In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,accuracy_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


### Import the dataset

In [ ]:
imdb_data = pd.read_csv('./IMDB_Dataset.csv')
imdb_data.head()

In [ ]:
imdb_data['sentiment'].value_counts()

### Generate the Training and Testing data by dividing the dataset as 4:1

In [ ]:
train_reviews=imdb_data.review[:40000]
train_sentiments=imdb_data.sentiment[:40000]

test_reviews=imdb_data.review[40000:]
test_sentiments=imdb_data.sentiment[40000:]

train_reviews.shape,train_sentiments.shape, test_reviews.shape,test_sentiments.shape


### Make sure that the split is balanced in general

In [ ]:
train_sentiments.value_counts()

### Preprocessing the dataset

In [ ]:
# tokenizer = ToktokTokenizer()
# stopword_list=nltk.corpus.stopwords.words('english')

# #Stemming the text, e.g. am, are, is -> be
# def simple_stemmer(text):
#     ps=nltk.porter.PorterStemmer()
#     text= ' '.join([ps.stem(word) for word in text.split()])
#     return text

# train_reviews = train_reviews.apply(simple_stemmer)
# test_reviews = test_reviews.apply(simple_stemmer)

In [ ]:
# set stopwords to english
# stop = set(stopwords.words('english'))
# print(stop)

In [ ]:
# #removing the stopwords
# def remove_stopwords(text, is_lower_case=False):
#     tokens = tokenizer.tokenize(text)
#     tokens = [token.strip() for token in tokens]
#     if is_lower_case:
#         filtered_tokens = [token for token in tokens if token not in stopword_list]
#     else:
#         filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
#     filtered_text = ' '.join(filtered_tokens)    
#     return filtered_text

# train_reviews = train_reviews.apply(remove_stopwords)
# test_reviews = test_reviews.apply(remove_stopwords)


如果在max_df中超过百分之50的一个词其中大部分都在positive，positive indicator？

 # Naive bayes model
 https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

## Method that take in the training dataset, then return the positive and negative words-log probability dictionary.


In [ ]:
'''
Method that take in the training dataset, then return the positive and negative words log probability.
Input: train_reviews: reviews (sentences) for training
       train_sentiments: sentiments (label) for training
       tfidf: boolean variable indicating whether using bow or tfidf
       alpha: laplance smoothing variable, default to be 1.0
       ngram_range: the scale of ngram model will be used, default = (1,1) unigram
return: negative_word_log_prob_dict: dictionary that contains the word:log probability pair for negative class
        positive_word_log_prob_dict: dictionary that contains the word:log probability pair for positive class
        mnb: the trained multinomial naive bayes model, later can be used for testing
        transformed_test_reviews: transformed test reviews that later can be used for testing
        vec: either the tfidfVectorize build from tfidf model or the CountVectorizer build from Bag of word model.
'''

def generate_log_prob(train_reviews, train_sentiments, tfidf=False, alpha=1.0, ngram_range = (1,1)):

    if (tfidf):
        #Tfidf vectorizer
        vec=TfidfVectorizer(use_idf=tfidf, ngram_range=ngram_range)
        #transformed train reviews
        transformed_train_reviews=vec.fit_transform(train_reviews)
        #transformed test reviews
        transformed_test_reviews=vec.transform(test_reviews)
    else:
        vec=CountVectorizer(ngram_range=(1,1))
        transformed_train_reviews=vec.fit_transform(train_reviews)
        transformed_test_reviews=vec.transform(test_reviews)

    #training the model
    mnb = MultinomialNB(alpha=alpha)

    #fitting the naive bayes for bag of words
    mnb = mnb.fit(transformed_train_reviews, train_sentiments)
    negative_log_prob = mnb.feature_log_prob_[0]
    positive_log_prob = mnb.feature_log_prob_[1]

    # Generate two dict: word:log_prob
    negative_word_log_prob_dict = {}
    positive_word_log_prob_dict = {}
    for word, index in vec.vocabulary_.items():
        negative_word_log_prob_dict[word] = negative_log_prob[index]
        positive_word_log_prob_dict[word] = positive_log_prob[index]
    
    return negative_word_log_prob_dict, positive_word_log_prob_dict, mnb, transformed_test_reviews, vec

## Method that given input word, manually change the weight of the word in naive bayes model

In [ ]:
'''
This method will take in a word:scale dict, then take in the negative and positive word:log_probability dict, manually change the weight of the words in the model and the dict
Input: word_change_scale: this is the word-scale dictionary, how much the weight of the word should be changed, For example if the value is 0.5, we will say 
                          the probability of the word in negative class should multiply 0.5, in original probability, we take power to the scale
       model: the trained naive bayes model, which the feature_log_prob_ attribute will be manually changed based on previous two params
       negative_word_log_prob_dict: dictionary that contains the word:log probability pair for negative class, which some values will be changed
       positive_word_log_prob_dict: dictionary that contains the word:log probability pair for positive class, which some values will be changed
       vec: either the tfidfVectorize build from tfidf model or the CountVectorizer build from Bag of word model.
return: negative_word_change_scale: The modified negative dict
        positive_word_change_scale: The modified positive dict
        model: The modified naive bayes model
'''

def change_weight(word_change_scale, model, negative_word_log_prob_dict, positive_word_log_prob_dict, vec):
    for word, scale in word_change_scale.items():
        # change the weight of words in negative and positive word:log_prob dict
        negative_word_log_prob_dict[word] *= scale
        positive_word_log_prob_dict[word] *= scale

        # change the weight of words in the model
        index_in_model = vec.vocabulary_[word]
        model.feature_log_prob_[0][index_in_model] *= scale
        model.feature_log_prob_[1][index_in_model] *= scale

    return negative_word_log_prob_dict, positive_word_log_prob_dict, model

### Predict the result using bag of word methods

In [ ]:
neg, pos, mnb, transformed_test_reviews, vec= generate_log_prob(train_reviews, train_sentiments, alpha = 0.05)

This block represent the length of the vocabulary

In [ ]:
len(vec.vocabulary_.items())

Following two blocks shows the word:log_prob key value pair for each phrase in the vocabulary

In [ ]:
pos

In [ ]:
neg

Following block use the trained model to do the prediction

In [ ]:
mnb_bow_predict = mnb.predict(transformed_test_reviews)


To check the corresponding test reviews log probability and original probability.


NOTICE: this probability was being normalized across all classes, so the log_probability was not the summation of log class probability and log probability of all phrases in the sentence. You can see that the summation of predict_proba() method is 1. The predict_log_proba() method calculate the log probability based on the normalized one.

In [ ]:
mnb.predict_proba(transformed_test_reviews[0]), mnb.predict_log_proba(transformed_test_reviews[0])

This predict() method will return the predict value of given transformed review

In [ ]:
mnb.predict(transformed_test_reviews[0]), mnb.predict(transformed_test_reviews[1])

This class_log_prior_ attribute of the MultinomialNB() model gives the prior class probability. In the order of the classes_ attribute

In [ ]:
mnb.classes_, mnb.class_log_prior_

Now we put the entire training set into the trained Naive Bayes model, and ge the score

In [ ]:
mnb_bow_predict = mnb.predict(transformed_test_reviews)
mnb_bow_score = accuracy_score(test_sentiments, mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)


Classification report

In [ ]:
#Classification report for bag of words 
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)

In [ ]:
wrong_labeled_NB_BOW = ""

for i in range(mnb_bow_predict.size):
    start_index = test_sentiments.index.start
    if(mnb_bow_predict[i] != test_sentiments[start_index + i]):
        wrong_labeled_NB_BOW += str(start_index + i) + " " + str(imdb_data['sentiment'][start_index + i]) + " | " + str(imdb_data['review'][start_index + i]) +  "\n"

f = open("Naive Bayes - Bag of Words Wrongly labeled sentences.txt", "w")
f.write(wrong_labeled_NB_BOW)
f.close()

### Predict the model using TF-IDF method

In [ ]:
neg, pos, mnb, transformed_test_reviews, vec = generate_log_prob(train_reviews, train_sentiments, tfidf = True, alpha = 0.05)

In [ ]:
mnb_tfidf_predict = mnb.predict(transformed_test_reviews)
mnb_tfidf_score = accuracy_score(test_sentiments, mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)


In [ ]:
#Classification report for TF-IDF
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

In [ ]:
wrong_labeled_NB_TFIDF = ""
for i in range(mnb_tfidf_predict.size):
    start_index = test_sentiments.index.start
    if(mnb_tfidf_predict[i] != test_sentiments[start_index + i]):
        wrong_labeled_NB_TFIDF += str(start_index + i) + " " + str(imdb_data['sentiment'][start_index + i]) + " | " + str(imdb_data['review'][start_index + i]) +  "\n"

f = open("Naive Bayes - TF-IDF Wrongly labeled sentences.txt", "w")
f.write(wrong_labeled_NB_TFIDF)
f.close()

# fastText method

In [ ]:
import fasttext
test_reviews.index.stop

In [ ]:
# #preprocess the data so it can feed into fasttext
# def transform_instance(doc, label):
#     processed_text = []
#     #Prefix the index-ed label with __label__
#     for i in range(doc.index.start, doc.index.stop):
#         cur_row = "__label__" + label[i] + " " + doc[i]
#         processed_text.append(cur_row)
#     return processed_text

# training_text = transform_instance(train_reviews, train_sentiments)
# test_text = transform_instance(test_reviews, test_sentiments)

# # Put the training and test dataset into file, so that the fasttext model can read them.
# f = open("fasttext.train", "w")
# output = ""
# for text in training_text:
#     output += text
#     output += "\n"
# f.write(output)
# f.close()

# f = open("fasttext.test", "w")
# output = ""
# for text in test_text:
#     output += text
#     output += "\n"
# f.write(output)
# f.close()

In [ ]:
model = fasttext.train_supervised(input = "fasttext.train", lr=0.1, epoch=45, wordNgrams=1)

In [ ]:
model.save_model("model_movie.bin")

In [ ]:
model.test("fasttext.test")


In [ ]:
# get the misclassified examples by fasttext algorithm
wrong_labeled_fasttext = ""
for i in range(test_reviews.size):
    start_index = test_sentiments.index.start
    if(model.predict(test_reviews[start_index + i])[0][0] != str('__label__' + test_sentiments[start_index + i])):
        wrong_labeled_fasttext += str(start_index + i) + " " + str(imdb_data['sentiment'][start_index + i]) + " | " + str(imdb_data['review'][start_index + i]) +  "\n"

f = open("fastText Wrongly labeled sentences.txt", "w")
f.write(wrong_labeled_fasttext)
f.close


# Fasttext manually modify the sentence embedding method

In [ ]:
train_reviews
import torch
import torch.nn as nn

In [ ]:
# def output_train_reviews_to_doc(train_reviews):
#     processed_doc = ""
#     for review in train_reviews:
#         processed_doc += review + "\n"
#     return processed_doc

# f = open("wordRepresentation.train", "w")
# f.write(output_train_reviews_to_doc(train_reviews))
# f.close()

In [ ]:
model = fasttext.train_unsupervised("wordRepresentation.train")


In [ ]:
len(model.words)

In [ ]:
model.get_word_vector("the")

For each train and test review, use all the embeddings of words, calculate an average to represent the sentence

In [ ]:
def transform_sentence_embedding(data, word_list):
    representations = []
    for sentence in data:
        nltk_tokens = nltk.word_tokenize(sentence)
        sum = 0
        count = 0
        for token in nltk_tokens:
            if token not in word_list:
                sum += model.get_word_vector(token)
                count += 1
        representations.append(sum / count)
    return representations



In [ ]:
def transform_label(data):
    representations = []
    for label in data:
        if(label == 'negative'):
            representations.append(0)
        else:
            representations.append(1)
    return representations

In [ ]:
word_list = []
fasttext_train_reviews = torch.Tensor(np.array(transform_sentence_embedding(train_reviews, word_list)))
fasttext_test_reviews = torch.Tensor(np.array(transform_sentence_embedding(test_reviews, word_list)))

In [ ]:
fasttext_train_sentiments = nn.functional.one_hot(torch.Tensor(np.array(transform_label(train_sentiments))).to(torch.int64)).to(torch.float)
fasttext_test_sentiments = nn.functional.one_hot(torch.Tensor(np.array(transform_label(test_sentiments))).to(torch.int64)).to(torch.float)

In [ ]:
fasttext_train_reviews.shape, fasttext_train_sentiments.shape, fasttext_test_reviews.shape, fasttext_test_sentiments.shape

In [ ]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 

        # Non-linearity
        self.sigmoid_1 = nn.Sigmoid()

        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

        # Non-linearity
        self.sigmoid_2 = nn.Sigmoid()

    def forward(self, x):
        # Linear function  # LINEAR
        out = self.fc1(x)

        # Non-linearity  # NON-LINEAR
        out = self.sigmoid_1(out)

        # Linear function (readout)  # LINEAR
        out = self.fc2(out)

        out = self.sigmoid_2(out)
        return out

In [ ]:
input_dim = 100
hidden_dim = 100
output_dim = 2

learning_rate = 0.1
n_epochs = 40
batch_size = 32

net = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)  


In [ ]:
net.train()
# Placeholder for loss
track_loss = []

print('Epoch', '\t', 'Loss train', '\t', 'Loss test')
for i in range(n_epochs):

    # shuffle the permutation, so that the batches each time are different
    shuffle_idx = np.random.permutation(len(fasttext_train_reviews))
    #split the tensor into smaller batches
    review_batches = torch.split(fasttext_train_reviews[shuffle_idx], batch_size)
    sentiment_batches = torch.split(fasttext_train_sentiments[shuffle_idx], batch_size)
    # Mini batches
    for j in range(len(review_batches)):
        review_batch = review_batches[j]

        sentiment_batch = sentiment_batches[j]
        
        #where we train and build the network
        output_train = net(review_batch)


        loss = criterion(output_train, sentiment_batch)
    
        # compute gradients
        optimizer.zero_grad()
        loss.backward()
    
        # update the weights
        optimizer.step()

    # Keep track of loss at each epoch
    track_loss += [float(loss)]

    loss_epoch = f'{i + 1} / {n_epochs}'
    with torch.no_grad():
        output_train = net(fasttext_train_reviews)
        loss_train = criterion(output_train, fasttext_train_sentiments)
        loss_epoch += f'\t {loss_train:.4f}'

        output_test = net(fasttext_test_reviews)
        loss_test = criterion(output_test, fasttext_test_sentiments)
        loss_epoch += f'\t\t {loss_test:.4f}'

    print(loss_epoch)

In [ ]:
predict_val = net(fasttext_test_reviews)

In [ ]:
predict_result = []
for (neg_prob, pos_prob) in predict_val:
    if (neg_prob > pos_prob):
        predict = "negative"
    else:
        predict = "positive"
    predict_result.append(predict)


In [ ]:
predict_result

In [ ]:
ffnn_score = accuracy_score(test_sentiments, predict_result)
ffnn_score